# Load FPL Data to CDF

This notebook fetches data from the Fantasy Premier League API and loads it into Cognite Data Fusion.

## 📋 What runs when:

### **One-time setup** (already done ✅):
- Create space, containers, views, and data model
- These persist in CDF and don't need to be recreated

### **Every time you want to update FPL data** (run weekly):
1. **Fetch FPL Data** - Gets latest stats from FPL API
2. **Load to CDF** - Updates your manager/gameweek data in CDF

### **Run the whole notebook:**
- It's safe to run all cells every time
- The setup cells are idempotent (they won't duplicate resources)
- Or skip to the "Fetch FPL Data" section if you just want to update scores

---


# Load Fantasy Premier League Data to CDF

This notebook fetches data from the FPL API and loads it into your CDF data model in the `fantasy_football` space.


In [88]:
import sys
sys.path.append('..')

import os
from datetime import datetime
from dotenv import load_dotenv
from cognite.client import CogniteClient
from cognite.client.config import ClientConfig
from cognite.client.credentials import OAuthClientCredentials
from cognite.client.data_classes.data_modeling import NodeApply, NodeOrEdgeData
from src.fpl_client import FPLClient

# Load environment variables
load_dotenv()

# Initialize CDF client
print("Connecting to CDF...")

# Get configuration from environment variables
cluster = os.getenv("CDF_CLUSTER", "bluefield")
project = os.getenv("CDF_PROJECT", "sofie-prod")
base_url = os.getenv("CDF_BASE_URL", f"https://{cluster}.cognitedata.com")
token_url = os.getenv("CDF_TOKEN_URL")
client_id = os.getenv("CDF_CLIENT_ID")
client_secret = os.getenv("CDF_CLIENT_SECRET")

# Validate required environment variables
if not all([token_url, client_id, client_secret]):
    raise ValueError(
        "Missing required environment variables. Please ensure your .env file contains:\n"
        "  - CDF_TOKEN_URL\n"
        "  - CDF_CLIENT_ID\n"
        "  - CDF_CLIENT_SECRET\n"
    )

creds = OAuthClientCredentials(
    token_url=token_url,
    client_id=client_id,
    client_secret=client_secret,
    scopes=[f"{base_url}/.default"],
)

cnf = ClientConfig(
    client_name="fantasy-football-client",
    project=project,
    credentials=creds,
    base_url=base_url,
)

client = CogniteClient(cnf)
print(f"✓ Connected to project: {client.config.project}")
print(f"✓ Cluster: {cluster}")

fpl_client = FPLClient()
print("✓ FPL API client initialized")
print("✓ FPL API client initialized")


Connecting to CDF...
✓ Connected to project: sofie-prod
✓ Cluster: bluefield
✓ FPL API client initialized
✓ FPL API client initialized


## Configuration


In [89]:
# Your FPL League ID
LEAGUE_ID = "1097811"  # Null digge damer igjen på Meny

# CDF Data Model settings
SPACE = "fantasy_football"
TEAM_VIEW = "Team"
GAMEWEEK_VIEW = "Gameweek"
MANAGER_VIEW = "Manager"
PERFORMANCE_VIEW = "ManagerGameweekPerformance"
VERSION = "1"

print(f"League ID: {LEAGUE_ID}")
print(f"CDF Space: {SPACE}")


League ID: 1097811
CDF Space: fantasy_football


## 1. Fetch Data from FPL API


In [90]:
print("Fetching bootstrap data (teams, gameweeks)...")
bootstrap = fpl_client.get_bootstrap_static()

teams = bootstrap['teams']
events = bootstrap['events']
current_gw = fpl_client.get_current_gameweek()

print(f"✓ Fetched {len(teams)} teams")
print(f"✓ Fetched {len(events)} gameweeks")
print(f"✓ Current gameweek: {current_gw}")


Fetching bootstrap data (teams, gameweeks)...
✓ Fetched 20 teams
✓ Fetched 38 gameweeks
✓ Current gameweek: 11


In [91]:
print(f"\nFetching league data for '{LEAGUE_ID}'...")
league_data = fpl_client.get_league_standings(LEAGUE_ID)

league_info = league_data['league']
standings = league_data['standings']['results']

print(f"✓ League: {league_info['name']}")
print(f"✓ Fetched {len(standings)} managers")



Fetching league data for '1097811'...
✓ League: Null digge damer igjen på Meny
✓ Fetched 27 managers


## 2. Load Teams to CDF


In [92]:
print("Creating Team nodes...")

team_nodes = []
for team in teams:
    node = NodeApply(
        space=SPACE,
        external_id=f"team_{team['id']}",
        sources=[
            NodeOrEdgeData(
                source={
                    "space": SPACE,
                    "externalId": "Team",
                    "version": VERSION,
                    "type": "view"
                },
                properties={
                    "teamId": team['id'],
                    "name": team['name'],
                    "shortName": team['short_name'],
                    "strength": team.get('strength')
                }
            )
        ]
    )
    team_nodes.append(node)

# Write to CDF
result = client.data_modeling.instances.apply(nodes=team_nodes, auto_create_direct_relations=True)
print(f"✓ Created/updated {len(team_nodes)} team nodes")
print(f"  Example: {teams[0]['name']}")


Creating Team nodes...
✓ Created/updated 20 team nodes
  Example: Arsenal


## 3. Load Gameweeks to CDF


In [93]:
print("Creating Gameweek nodes...")

gameweek_nodes = []
for event in events:
    # Parse deadline time
    deadline = None
    if event.get('deadline_time'):
        try:
            deadline = datetime.fromisoformat(event['deadline_time'].replace('Z', '+00:00'))
        except:
            pass
    
    node = NodeApply(
        space=SPACE,
        external_id=f"gameweek_{event['id']}",
        sources=[
            NodeOrEdgeData(
                source={
                    "space": SPACE,
                    "externalId": "Gameweek",
                    "version": VERSION,
                    "type": "view"
                },
                properties={
                    "gameweekNumber": event['id'],
                    "name": event['name'],
                    "deadlineTime": deadline,
                    "isFinished": event['finished'],
                    "isCurrent": event.get('is_current', False),
                    "averageScore": event.get('average_entry_score'),
                    "highestScore": event.get('highest_score')
                }
            )
        ]
    )
    gameweek_nodes.append(node)

# Write to CDF
result = client.data_modeling.instances.apply(nodes=gameweek_nodes, auto_create_direct_relations=True)
print(f"✓ Created/updated {len(gameweek_nodes)} gameweek nodes")
print(f"  Current: Gameweek {current_gw}")


Creating Gameweek nodes...
✓ Created/updated 38 gameweek nodes
  Current: Gameweek 11


## 4. Load Managers to CDF


In [94]:
print("Creating Manager nodes...")

manager_nodes = []
for manager in standings:
    node = NodeApply(
        space=SPACE,
        external_id=f"manager_{manager['entry']}",
        sources=[
            NodeOrEdgeData(
                source={
                    "space": SPACE,
                    "externalId": "Manager",
                    "version": VERSION,
                    "type": "view"
                },
                properties={
                    "entryId": manager['entry'],
                    "managerName": manager['player_name'],
                    "teamName": manager['entry_name'],
                    "overallPoints": manager['total'],
                    "overallRank": manager.get('rank'),
                    "leagueRank": manager.get('rank'),
                    "teamValue": None  # Will be updated with detailed data
                }
            )
        ]
    )
    manager_nodes.append(node)

# Write to CDF
result = client.data_modeling.instances.apply(nodes=manager_nodes, auto_create_direct_relations=True)
print(f"✓ Created/updated {len(manager_nodes)} manager nodes")
print(f"  Managers: {', '.join([m['player_name'] for m in standings[:3]])}, ...")


Creating Manager nodes...
✓ Created/updated 27 manager nodes
  Managers: edvard ekberg, Andreas Horten Jespersen, Andreas Røgeberg, ...


## 5. Load Manager Performance History to CDF


In [95]:
print("Fetching and creating ManagerGameweekPerformance nodes...")
print("This will fetch history for each manager (may take a minute)...\n")

performance_nodes = []
managers_processed = 0

for manager in standings:
    entry_id = manager['entry']
    manager_name = manager['player_name']
    
    try:
        print(f"  {managers_processed + 1}/{len(standings)}: {manager_name}...", end=" ")
        
        # Fetch history for this manager
        history = fpl_client.get_entry_history(entry_id)
        
        # Process each gameweek
        for gw_data in history.get('current', []):
            gameweek = gw_data['event']
            
            node = NodeApply(
                space=SPACE,
                external_id=f"performance_{entry_id}_gw{gameweek}",
                sources=[
                    NodeOrEdgeData(
                        source={
                            "space": SPACE,
                            "externalId": "ManagerGameweekPerformance",
                            "version": VERSION,
                            "type": "view"
                        },
                        properties={
                            "manager": {
                                "space": SPACE,
                                "externalId": f"manager_{entry_id}"
                            },
                            "gameweek": {
                                "space": SPACE,
                                "externalId": f"gameweek_{gameweek}"
                            },
                            "points": gw_data['points'],
                            "totalPoints": gw_data['total_points'],
                            "rank": gw_data.get('overall_rank'),
                            "gameweekRank": gw_data.get('rank'),
                            "transfers": gw_data.get('event_transfers', 0),
                            "transferCost": gw_data.get('event_transfers_cost', 0),
                            "bank": gw_data.get('bank', 0) / 10.0,
                            "teamValue": gw_data.get('value', 0) / 10.0
                        }
                    )
                ]
            )
            performance_nodes.append(node)
        
        managers_processed += 1
        print(f"✓ {len(history.get('current', []))} gameweeks")
        
    except Exception as e:
        print(f"✗ Error: {e}")
        continue

print(f"\nWriting {len(performance_nodes)} performance records to CDF...")
# Write in batches of 100
batch_size = 100
for i in range(0, len(performance_nodes), batch_size):
    batch = performance_nodes[i:i + batch_size]
    client.data_modeling.instances.apply(nodes=batch, auto_create_direct_relations=True)
    print(f"  Batch {i//batch_size + 1}/{(len(performance_nodes)-1)//batch_size + 1} written")

print(f"\n✓ Created/updated {len(performance_nodes)} performance records")


Fetching and creating ManagerGameweekPerformance nodes...
This will fetch history for each manager (may take a minute)...

  1/27: edvard ekberg... ✓ 11 gameweeks
  2/27: Andreas Horten Jespersen... ✓ 11 gameweeks
  3/27: Andreas Røgeberg... ✓ 11 gameweeks
  4/27: Martin Tørhaug... ✓ 11 gameweeks
  5/27: Stian Røgeberg... ✓ 11 gameweeks
  6/27: Håkon Kornmo... ✓ 11 gameweeks
  7/27: Arnstein Nisja... ✓ 11 gameweeks
  8/27: Even H. Larsen... ✓ 11 gameweeks
  9/27: Hakon Ness... ✓ 11 gameweeks
  10/27: Mathias Moe... ✓ 11 gameweeks
  11/27: Julian Dahlskås... ✓ 11 gameweeks
  12/27: Magnus Urheim... ✓ 11 gameweeks
  13/27: Benjamin Aas Hansen... ✓ 11 gameweeks
  14/27: Mats Kjenes... ✓ 11 gameweeks
  15/27: Vetle Skaar Eriksen... ✓ 11 gameweeks
  16/27: Hamed Dawlatshahi... ✓ 11 gameweeks
  17/27: Jonatan Syrdahl Steen... ✓ 11 gameweeks
  18/27: Helle Elvebakk... ✓ 11 gameweeks
  19/27: Jakob Brunvatne... ✓ 11 gameweeks
  20/27: Andreas Braatun... ✓ 11 gameweeks
  21/27: Andreas Bruun...

## 6. Load Players to CDF

Fetch all FPL players and their stats


In [96]:
print("Creating Player nodes...")
player_nodes = []
players = bootstrap['elements']  # All FPL players

position_map = {1: "GK", 2: "DEF", 3: "MID", 4: "FWD"}

for player in players:
    player_nodes.append(NodeApply(
        space=SPACE,
        external_id=f"player_{player['id']}",
        sources=[
            NodeOrEdgeData(
                source={
                    "space": SPACE,
                    "externalId": "Player",
                    "version": VERSION,
                    "type": "view"
                },
                properties={
                    "playerId": player['id'],
                    "webName": player['web_name'],
                    "firstName": player['first_name'],
                    "lastName": player['second_name'],
                    "team": {"space": SPACE, "externalId": f"team_{player['team']}"},
                    "position": position_map.get(player['element_type'], "Unknown"),
                    "currentPrice": player['now_cost'] / 10.0,  # Convert to millions
                    "totalPoints": player['total_points'],
                    "form": float(player.get('form', 0)) if player.get('form') else 0.0,
                    "selectedByPercent": float(player.get('selected_by_percent', 0)) if player.get('selected_by_percent') else 0.0,
                    "pointsPerGame": float(player.get('points_per_game', 0)) if player.get('points_per_game') else 0.0
                }
            )
        ]
    ))

if player_nodes:
    result = client.data_modeling.instances.apply(nodes=player_nodes, auto_create_direct_relations=True)
    print(f"✓ Loaded {len(player_nodes)} players to CDF")
else:
    print("⚠️  No players to load")


Creating Player nodes...
✓ Loaded 752 players to CDF


In [97]:
print("Fetching manager histories for analytics...")
manager_histories = {}

for manager in standings:
    entry_id = manager['entry']
    try:
        history = fpl_client.get_entry_history(entry_id)
        manager_histories[entry_id] = history
    except Exception as e:
        print(f"  Warning: Could not fetch history for manager {entry_id}: {e}")
        
print(f"✓ Fetched histories for {len(manager_histories)} managers")


Fetching manager histories for analytics...
✓ Fetched histories for 27 managers


## 7. Compute Manager Analytics

Calculate consistency scores, transfer success rates, and team value growth


In [98]:
import numpy as np

print("Computing manager analytics...")
manager_analytics = {}

for entry_id, history_data in manager_histories.items():
    current_gw_data = history_data.get('current', [])
    
    if not current_gw_data:
        continue
    
    # Calculate weekly points
    weekly_points = [gw['points'] for gw in current_gw_data]
    
    # Consistency Score (0-100): based on coefficient of variation (inverse)
    # Lower std dev relative to mean = more consistent = higher score
    if len(weekly_points) > 1 and np.mean(weekly_points) > 0:
        points_mean = np.mean(weekly_points)
        points_std = np.std(weekly_points)
        coeff_variation = points_std / points_mean
        # Convert to 0-100 scale (lower CV = higher score)
        consistency_score = max(0, min(100, 100 * (1 - min(coeff_variation, 1))))
    else:
        consistency_score = 0.0
        points_mean = np.mean(weekly_points) if weekly_points else 0.0
        points_std = 0.0
    
    # Team value growth
    if current_gw_data:
        starting_value = current_gw_data[0]['value'] / 10.0 if current_gw_data else 100.0
        current_value = current_gw_data[-1]['value'] / 10.0 if current_gw_data else 100.0
        team_value_growth = current_value - starting_value
    else:
        team_value_growth = 0.0
    
    # Count transfers
    total_transfers = sum(gw.get('event_transfers', 0) for gw in current_gw_data)
    
    manager_analytics[entry_id] = {
        'consistencyScore': round(consistency_score, 2),
        'averagePointsPerWeek': round(points_mean, 2),
        'pointsStdDev': round(points_std, 2),
        'teamValueGrowth': round(team_value_growth, 2),
        'totalTransfers': total_transfers,
        # Transfer success will be calculated after analyzing actual transfers
        'transferSuccessRate': 0.0,
        'successfulTransfers': 0
    }

print(f"✓ Computed analytics for {len(manager_analytics)} managers")
print(f"\nExample analytics (first manager):")
if manager_analytics:
    example_entry = list(manager_analytics.keys())[0]
    print(f"  Entry {example_entry}:")
    for key, value in manager_analytics[example_entry].items():
        print(f"    {key}: {value}")


Computing manager analytics...
✓ Computed analytics for 27 managers

Example analytics (first manager):
  Entry 653439:
    consistencyScore: 81.36
    averagePointsPerWeek: 62.18
    pointsStdDev: 11.59
    teamValueGrowth: 2.8
    totalTransfers: 9
    transferSuccessRate: 0.0
    successfulTransfers: 0


In [99]:
print("Updating Manager nodes with analytics...")
updated_manager_nodes = []

for standing in standings:
    entry_id = standing['entry']
    history_data = manager_histories.get(entry_id, {})
    analytics = manager_analytics.get(entry_id, {})
    
    if history_data.get('current'):
        current_data = history_data['current'][0]
        
        updated_manager_nodes.append(NodeApply(
            space=SPACE,
            external_id=f"manager_{entry_id}",
            sources=[
                NodeOrEdgeData(
                    source={
                        "space": SPACE,
                        "externalId": "Manager",
                        "version": VERSION,
                        "type": "view"
                    },
                    properties={
                        "entryId": entry_id,
                        "managerName": standing["player_name"],
                        "teamName": standing["entry_name"],
                        "overallPoints": current_data["total_points"],
                        "overallRank": current_data.get("rank", 0),
                        "leagueRank": standing["rank"],
                        "teamValue": current_data["value"] / 10.0,
                        # Add new analytics
                        "consistencyScore": analytics.get('consistencyScore', 0.0),
                        "averagePointsPerWeek": analytics.get('averagePointsPerWeek', 0.0),
                        "pointsStdDev": analytics.get('pointsStdDev', 0.0),
                        "teamValueGrowth": analytics.get('teamValueGrowth', 0.0),
                        "totalTransfers": analytics.get('totalTransfers', 0),
                        "transferSuccessRate": analytics.get('transferSuccessRate', 0.0),
                        "successfulTransfers": analytics.get('successfulTransfers', 0)
                    }
                )
            ]
        ))

if updated_manager_nodes:
    result = client.data_modeling.instances.apply(nodes=updated_manager_nodes, auto_create_direct_relations=True)
    print(f"✓ Updated {len(updated_manager_nodes)} managers with analytics")
else:
    print("⚠️  No managers to update")


Updating Manager nodes with analytics...
✓ Updated 27 managers with analytics


## 8. Analyze Manager Team Betting Patterns

Compute which Premier League teams each manager is good at picking players from


In [112]:
from collections import defaultdict
import time

print("Analyzing manager team betting patterns (FIXED VERSION)...")
print("(This may take a few minutes as we fetch picks for each manager/gameweek)")

# Create a mapping of player_id -> team_id from players data
player_team_map = {player['id']: player['team'] for player in players}

# Create a dictionary of teams (teams is a list, convert to dict by id)
teams_dict = {team['id']: team for team in teams}

# Create player stats lookup - use form as proxy for gameweek points
player_stats = {}
for player in players:
    player_id = player['id']
    player_stats[player_id] = {
        'form': float(player.get('form', '0')),  # Average points per game (last few GWs)
        'ppg': float(player.get('points_per_game', '0')),
        'total_points': player.get('total_points', 0)
    }

# Store team betting stats for each manager
manager_team_stats = defaultdict(lambda: defaultdict(lambda: {
    'players_used': set(),
    'total_points': 0,
    'gameweeks': set(),
    'player_appearances': 0  # Count of player-gameweek combinations
}))

# Fetch picks for recent gameweeks (last 5 to keep it manageable)
recent_gameweeks = range(max(1, current_gw - 4), current_gw + 1)

managers_processed = 0
for entry_id in [s['entry'] for s in standings]:
    print(f"  Processing manager {entry_id}...", end=" ")
    manager_points = 0
    
    for gw in recent_gameweeks:
        try:
            picks_data = fpl_client.get_entry_picks(entry_id, gw)
            picks = picks_data.get('picks', [])
            
            # Get total gameweek points for this manager
            entry_history = picks_data.get('entry_history', {})
            gw_total_points = entry_history.get('points', 0)
            
            # Calculate total expected form for proportional distribution
            total_form = 0
            for pick in picks:
                player_id = pick['element']
                if player_id in player_stats:
                    form = player_stats[player_id]['form']
                    multiplier = pick['multiplier']
                    total_form += form * multiplier
            
            # Distribute points to teams based on player form
            for pick in picks:
                player_id = pick['element']
                team_id = player_team_map.get(player_id)
                multiplier = pick['multiplier']
                
                if team_id and player_id in player_stats:
                    # Use form to estimate this player's contribution
                    player_form = player_stats[player_id]['form'] * multiplier
                    
                    # Distribute total gameweek points proportionally
                    if total_form > 0:
                        estimated_points = (player_form / total_form) * gw_total_points
                    else:
                        # Fallback: use form directly if no total
                        estimated_points = player_form
                    
                    manager_team_stats[entry_id][team_id]['players_used'].add(player_id)
                    manager_team_stats[entry_id][team_id]['total_points'] += estimated_points
                    manager_team_stats[entry_id][team_id]['gameweeks'].add(gw)
                    manager_team_stats[entry_id][team_id]['player_appearances'] += 1
                    
                    manager_points += estimated_points
            
            time.sleep(0.3)  # Rate limiting
            
        except Exception as e:
            print(f"\\n    Warning: Could not fetch picks for GW{gw}: {e}")
            continue
    
    print(f"✓ {manager_points:.0f} total points")
    managers_processed += 1

print(f"\\n✓ Analyzed team betting patterns for {managers_processed} managers")

# Show sample results
print("\\nSample Results (first 3 managers, top 5 teams each):")
for entry_id in list(manager_team_stats.keys())[:3]:
    manager_name = next((s['player_name'] for s in standings if s['entry'] == entry_id), f"Manager {entry_id}")
    print(f"\\n{manager_name}:")
    sorted_teams = sorted(manager_team_stats[entry_id].items(), 
                         key=lambda x: x[1]['total_points'], reverse=True)
    for team_id, stats in sorted_teams[:5]:
        # Use teams_dict instead of teams.get()
        team_name = teams_dict.get(team_id, {}).get('name', f'Team {team_id}')
        avg_pts = stats['total_points'] / len(stats['players_used']) if stats['players_used'] else 0
        print(f"  {team_name:20s}: {stats['total_points']:6.1f} pts from {len(stats['players_used'])} players (avg: {avg_pts:.1f})")

Analyzing manager team betting patterns (FIXED VERSION)...
(This may take a few minutes as we fetch picks for each manager/gameweek)
  Processing manager 653439... ✓ 309 total points
  Processing manager 2589656... ✓ 322 total points
  Processing manager 2078000... ✓ 308 total points
  Processing manager 7470993... ✓ 321 total points
  Processing manager 290121... ✓ 331 total points
  Processing manager 7669946... ✓ 296 total points
  Processing manager 4203952... ✓ 330 total points
  Processing manager 3550350... ✓ 300 total points
  Processing manager 4930626... ✓ 307 total points
  Processing manager 631108... ✓ 319 total points
  Processing manager 1251952... ✓ 304 total points
  Processing manager 1278950... ✓ 309 total points
  Processing manager 2635754... ✓ 308 total points
  Processing manager 5409736... ✓ 326 total points
  Processing manager 9083472... ✓ 278 total points
  Processing manager 6576934... ✓ 286 total points
  Processing manager 732306... ✓ 299 total points
  Pr

In [113]:
print("Analyzing transfers for each manager...")
print("This will fetch picks for each gameweek and compare to find transfers\n")

# Get all players for lookup
players_by_id = {p['id']: p for p in bootstrap.get('elements', [])}

# Store all transfers
all_transfers = []
managers_processed = 0

for manager in standings:
    entry_id = manager['entry']
    manager_name = manager['player_name']
    
    try:
        print(f"  {managers_processed + 1}/{len(standings)}: {manager_name}...", end=" ")
        
        # Get history to know which gameweeks to fetch
        history = fpl_client.get_entry_history(entry_id)
        gameweeks = [gw['event'] for gw in history.get('current', [])]
        
        # Fetch picks for each gameweek
        picks_by_gw = {}
        for gw in gameweeks:
            try:
                picks_data = fpl_client.get_entry_picks(entry_id, gw)
                picks_by_gw[gw] = {
                    'picks': picks_data.get('picks', []),
                    'transfers': picks_data.get('entry_history', {})
                }
            except:
                continue
        
        # Compare consecutive gameweeks to find transfers
        sorted_gws = sorted(picks_by_gw.keys())
        for i in range(len(sorted_gws) - 1):
            prev_gw = sorted_gws[i]
            curr_gw = sorted_gws[i + 1]
            
            prev_squad = {pick['element'] for pick in picks_by_gw[prev_gw]['picks']}
            curr_squad = {pick['element'] for pick in picks_by_gw[curr_gw]['picks']}
            
            # Find players in and out
            players_in = curr_squad - prev_squad
            players_out = prev_squad - curr_squad
            
            # Match transfers (assume same number in and out)
            if players_in and players_out:
                players_in_list = list(players_in)
                players_out_list = list(players_out)
                
                # Get transfer cost for this gameweek
                transfer_cost = picks_by_gw[curr_gw]['transfers'].get('event_transfers_cost', 0)
                num_transfers = picks_by_gw[curr_gw]['transfers'].get('event_transfers', 0)
                cost_per_transfer = transfer_cost / num_transfers if num_transfers > 0 else 0
                
                # Create transfer pairs
                for player_in_id, player_out_id in zip(players_in_list, players_out_list):
                    player_in = players_by_id.get(player_in_id, {})
                    player_out = players_by_id.get(player_out_id, {})
                    
                    # Calculate points gained over next 3 gameweeks
                    # For simplicity, we'll use current season points as a proxy
                    # In a full implementation, you'd fetch player gameweek history
                    points_in = player_in.get('total_points', 0)
                    points_out = player_out.get('total_points', 0)
                    
                    # Estimate points in next 3 GWs based on form
                    form_in = float(player_in.get('form', 0))
                    form_out = float(player_out.get('form', 0))
                    
                    estimated_points_in = form_in * 3 if form_in else 0
                    estimated_points_out = form_out * 3 if form_out else 0
                    
                    net_benefit = estimated_points_in - estimated_points_out
                    was_successful = net_benefit > cost_per_transfer
                    
                    all_transfers.append({
                        'entry_id': entry_id,
                        'gameweek': curr_gw,
                        'player_in_id': player_in_id,
                        'player_out_id': player_out_id,
                        'player_in_price': player_in.get('now_cost', 0) / 10.0,
                        'player_out_price': player_out.get('now_cost', 0) / 10.0,
                        'transfer_cost': int(cost_per_transfer),
                        'points_gained_next_3gw': int(round(net_benefit)),
                        'net_benefit': int(round(net_benefit)),
                        'was_successful': was_successful
                    })
        
        managers_processed += 1
        print(f"✓ Found {len([t for t in all_transfers if t['entry_id'] == entry_id])} transfers")
        
    except Exception as e:
        print(f"✗ Error: {e}")
        managers_processed += 1
        continue

print(f"\n✓ Analyzed {len(all_transfers)} total transfers from {managers_processed} managers")

# Show some statistics
if all_transfers:
    successful = sum(1 for t in all_transfers if t['was_successful'])
    success_rate = (successful / len(all_transfers) * 100) if all_transfers else 0
    avg_benefit = sum(t['net_benefit'] for t in all_transfers) / len(all_transfers)
    
    print(f"\nTransfer Statistics:")
    print(f"  Total transfers: {len(all_transfers)}")
    print(f"  Successful: {successful} ({success_rate:.1f}%)")
    print(f"  Average net benefit: {avg_benefit:.1f} points")


Analyzing transfers for each manager...
This will fetch picks for each gameweek and compare to find transfers

  1/27: edvard ekberg... ✓ Found 19 transfers
  2/27: Andreas Horten Jespersen... ✓ Found 42 transfers
  3/27: Andreas Røgeberg... ✓ Found 22 transfers
  4/27: Martin Tørhaug... ✓ Found 17 transfers
  5/27: Stian Røgeberg... ✓ Found 21 transfers
  6/27: Håkon Kornmo... ✓ Found 17 transfers
  7/27: Arnstein Nisja... ✓ Found 11 transfers
  8/27: Even H. Larsen... ✓ Found 28 transfers
  9/27: Hakon Ness... ✓ Found 22 transfers
  10/27: Mathias Moe... ✓ Found 15 transfers
  11/27: Julian Dahlskås... ✓ Found 15 transfers
  12/27: Magnus Urheim... ✓ Found 37 transfers
  13/27: Benjamin Aas Hansen... ✓ Found 20 transfers
  14/27: Mats Kjenes... ✓ Found 12 transfers
  15/27: Vetle Skaar Eriksen... ✓ Found 4 transfers
  16/27: Hamed Dawlatshahi... ✓ Found 17 transfers
  17/27: Jonatan Syrdahl Steen... ✓ Found 20 transfers
  18/27: Helle Elvebakk... ✓ Found 21 transfers
  19/27: Jakob B

In [114]:
print("Creating Transfer nodes...")

transfer_nodes = []

for transfer in all_transfers:
    entry_id = transfer['entry_id']
    gw = transfer['gameweek']
    player_in_id = transfer['player_in_id']
    player_out_id = transfer['player_out_id']
    
    transfer_nodes.append(NodeApply(
        space=SPACE,
        external_id=f"transfer_{entry_id}_gw{gw}_{player_out_id}to{player_in_id}",
        sources=[
            NodeOrEdgeData(
                source={
                    "space": SPACE,
                    "externalId": "Transfer",
                    "version": VERSION,
                    "type": "view"
                },
                properties={
                    "manager": {"space": SPACE, "externalId": f"manager_{entry_id}"},
                    "gameweek": {"space": SPACE, "externalId": f"gameweek_{gw}"},
                    "playerIn": {"space": SPACE, "externalId": f"player_{player_in_id}"},
                    "playerOut": {"space": SPACE, "externalId": f"player_{player_out_id}"},
                    "transferCost": transfer['transfer_cost'],
                    "playerInPrice": transfer['player_in_price'],
                    "playerOutPrice": transfer['player_out_price'],
                    "pointsGainedNext3GW": transfer['points_gained_next_3gw'],
                    "wasSuccessful": transfer['was_successful'],
                    "netBenefit": transfer['net_benefit']
                }
            )
        ]
    ))

if transfer_nodes:
    # Write in batches
    batch_size = 100
    for i in range(0, len(transfer_nodes), batch_size):
        batch = transfer_nodes[i:i + batch_size]
        client.data_modeling.instances.apply(nodes=batch, auto_create_direct_relations=True)
        print(f"  Batch {i//batch_size + 1}/{(len(transfer_nodes)-1)//batch_size + 1} written ({len(batch)} transfers)")
    
    print(f"\n✓ Loaded {len(transfer_nodes)} transfer records to CDF")
else:
    print("⚠️  No transfer data to load")


Creating Transfer nodes...
  Batch 1/6 written (100 transfers)
  Batch 2/6 written (100 transfers)
  Batch 3/6 written (100 transfers)
  Batch 4/6 written (100 transfers)
  Batch 5/6 written (100 transfers)
  Batch 6/6 written (10 transfers)

✓ Loaded 510 transfer records to CDF


In [120]:
print("Creating ManagerTeamBetting nodes...")
team_betting_nodes = []

for entry_id, team_stats in manager_team_stats.items():
    for team_id, stats in team_stats.items():
        total_players = len(stats['players_used'])
        total_points = stats['total_points']
        num_gameweeks = len(stats['gameweeks'])
        player_appearances = stats.get('player_appearances', total_players * num_gameweeks)
        
        if total_players == 0:
            continue
        
        # FIXED: Use player_appearances instead of total_players * num_gameweeks
        # This accounts for the actual number of times players from this team were picked
        avg_points_per_player = total_points / player_appearances if player_appearances > 0 else 0
        
        team_betting_nodes.append(NodeApply(
            space=SPACE,
            external_id=f"betting_{entry_id}_team_{team_id}",
            sources=[
                NodeOrEdgeData(
                    source={
                        "space": SPACE,
                        "externalId": "ManagerTeamBetting",
                        "version": VERSION,
                        "type": "view"
                    },
                    properties={
                        "manager": {"space": SPACE, "externalId": f"manager_{entry_id}"},
                        "team": {"space": SPACE, "externalId": f"team_{team_id}"},
                        "totalPlayersUsed": total_players,
                        "totalPoints": int(total_points),  # Convert to int
                        "averagePointsPerPlayer": round(avg_points_per_player, 2),
                        "successRate": round(min(100, (total_points / (player_appearances * 2)) * 100), 2) if player_appearances > 0 else 0.0,
                        "investmentValue": 0.0,  # Would need price data
                        "returnOnInvestment": 0.0  # Would need investment value
                    }
                )
            ]
        ))

if team_betting_nodes:
    result = client.data_modeling.instances.apply(nodes=team_betting_nodes, auto_create_direct_relations=True)
    print(f"✓ Loaded {len(team_betting_nodes)} team betting records to CDF")
    
    # Show sample of what was written
    print("\\nSample records written to CDF:")
    for node in team_betting_nodes[:3]:
        props = node.sources[0].properties
        team_id = props['team']['externalId'].replace('team_', '')
        team_name = teams_dict.get(int(team_id), {}).get('name', f'Team {team_id}')
        print(f"  {team_name}: {props['totalPoints']} points, {props['averagePointsPerPlayer']:.2f} avg")
else:
    print("⚠️  No team betting data to write")

Creating ManagerTeamBetting nodes...
✓ Loaded 100 team betting records to CDF
\nSample records written to CDF:
  Arsenal: 92 points, 6.16 avg
  Bournemouth: 28 points, 2.86 avg
  Everton: 7 points, 3.74 avg


## 9. Summary

**Note on Transfer Analysis:** 
Transfer success analysis requires tracking specific player transfers and comparing performance over subsequent gameweeks. This is complex and would significantly increase API calls. The `Transfer` view is available in the model for future enhancement.

For now, you have:
- ✅ **Player data** - All FPL players with stats
- ✅ **Manager analytics** - Consistency scores, team value growth, transfer counts
- ✅ **Team betting patterns** - Which PL teams managers pick from and their success rates
- 🔜 **Transfer analysis** - Can be added later with more detailed tracking


In [121]:
print("Fetching and creating ManagerGameweekPerformance nodes...")
print("This will fetch history for each manager (may take a minute)...\n")

performance_nodes = []
managers_processed = 0

for manager in standings:
    entry_id = manager['entry']
    manager_name = manager['player_name']
    
    try:
        print(f"  {managers_processed + 1}/{len(standings)}: {manager_name}...", end=" ")
        
        # Fetch history for this manager
        history = fpl_client.get_entry_history(entry_id)
        
        # Process each gameweek
        for gw_data in history.get('current', []):
            gameweek = gw_data['event']
            
            node = NodeApply(
                space=SPACE,
                external_id=f"performance_{entry_id}_gw{gameweek}",
                sources=[
                    NodeOrEdgeData(
                        source={
                            "space": SPACE,
                            "externalId": "ManagerGameweekPerformance",
                            "version": VERSION,
                            "type": "view"
                        },
                        properties={
                            "manager": {
                                "space": SPACE,
                                "externalId": f"manager_{entry_id}"
                            },
                            "gameweek": {
                                "space": SPACE,
                                "externalId": f"gameweek_{gameweek}"
                            },
                            "points": gw_data['points'],
                            "totalPoints": gw_data['total_points'],
                            "rank": gw_data.get('overall_rank'),
                            "gameweekRank": gw_data.get('rank'),
                            "transfers": gw_data.get('event_transfers', 0),
                            "transferCost": gw_data.get('event_transfers_cost', 0),
                            "bank": gw_data.get('bank', 0) / 10.0,  # Convert to millions
                            "teamValue": gw_data.get('value', 0) / 10.0  # Convert to millions
                        }
                    )
                ]
            )
            performance_nodes.append(node)
        
        managers_processed += 1
        print(f"✓ {len(history.get('current', []))} gameweeks")
        
    except Exception as e:
        print(f"✗ Error: {e}")
        continue

print(f"\nWriting {len(performance_nodes)} performance records to CDF...")
# Write in batches of 100
batch_size = 100
for i in range(0, len(performance_nodes), batch_size):
    batch = performance_nodes[i:i + batch_size]
    client.data_modeling.instances.apply(nodes=batch, auto_create_direct_relations=True)
    print(f"  Batch {i//batch_size + 1}/{(len(performance_nodes)-1)//batch_size + 1} written")

print(f"\n✓ Created/updated {len(performance_nodes)} performance records")


Fetching and creating ManagerGameweekPerformance nodes...
This will fetch history for each manager (may take a minute)...

  1/27: edvard ekberg... ✓ 11 gameweeks
  2/27: Andreas Horten Jespersen... ✓ 11 gameweeks
  3/27: Andreas Røgeberg... ✓ 11 gameweeks
  4/27: Martin Tørhaug... ✓ 11 gameweeks
  5/27: Stian Røgeberg... ✓ 11 gameweeks
  6/27: Håkon Kornmo... ✓ 11 gameweeks
  7/27: Arnstein Nisja... ✓ 11 gameweeks
  8/27: Even H. Larsen... ✓ 11 gameweeks
  9/27: Hakon Ness... ✓ 11 gameweeks
  10/27: Mathias Moe... ✓ 11 gameweeks
  11/27: Julian Dahlskås... ✓ 11 gameweeks
  12/27: Magnus Urheim... ✓ 11 gameweeks
  13/27: Benjamin Aas Hansen... ✓ 11 gameweeks
  14/27: Mats Kjenes... ✓ 11 gameweeks
  15/27: Vetle Skaar Eriksen... ✓ 11 gameweeks
  16/27: Hamed Dawlatshahi... ✓ 11 gameweeks
  17/27: Jonatan Syrdahl Steen... ✓ 11 gameweeks
  18/27: Helle Elvebakk... ✓ 11 gameweeks
  19/27: Jakob Brunvatne... ✓ 11 gameweeks
  20/27: Andreas Braatun... ✓ 11 gameweeks
  21/27: Andreas Bruun...

## 6. Summary & Verification


In [105]:
print("="*80)
print("DATA LOAD SUMMARY")
print("="*80)
print(f"✓ Teams: {len(team_nodes)} nodes")
print(f"✓ Gameweeks: {len(gameweek_nodes)} nodes")
print(f"✓ Managers: {len(manager_nodes)} nodes")
print(f"✓ Performance Records: {len(performance_nodes)} nodes")
print(f"\nTotal nodes created/updated: {len(team_nodes) + len(gameweek_nodes) + len(manager_nodes) + len(performance_nodes)}")
print("="*80)


DATA LOAD SUMMARY
✓ Teams: 20 nodes
✓ Gameweeks: 38 nodes
✓ Managers: 27 nodes
✓ Performance Records: 297 nodes

Total nodes created/updated: 382


In [116]:
print("\nVerifying data in CDF...")

# Query all nodes in the space
all_instances = client.data_modeling.instances.list(
    instance_type="node",
    space=SPACE,
    limit=200
)

print(f"✓ Found {len(all_instances)} total nodes in {SPACE} space")

# Analyze what we got
manager_nodes = []
gameweek_nodes = []
team_nodes = []
performance_nodes = []

for node in all_instances:
    # Check the external_id to determine node type
    ext_id = node.external_id
    
    if ext_id.startswith('manager_'):
        manager_nodes.append(node)
    elif ext_id.startswith('gameweek_'):
        gameweek_nodes.append(node)
    elif ext_id.startswith('team_'):
        team_nodes.append(node)
    elif ext_id.startswith('performance_'):
        performance_nodes.append(node)

print(f"\nNode breakdown:")
print(f"  - Teams: {len(team_nodes)}")
print(f"  - Gameweeks: {len(gameweek_nodes)}")
print(f"  - Managers: {len(manager_nodes)}")
print(f"  - Performance records: {len(performance_nodes)}")

# Display manager details
if manager_nodes:
    print(f"\n✓ Manager details:")
    for manager in manager_nodes:
        try:
            # Access properties using the correct structure
            if hasattr(manager, 'properties') and manager.properties:
                props = manager.properties.get(SPACE, {}).get(MANAGER_VIEW, {}).get(VERSION, {})
                if props:
                    name = props.get('managerName', 'Unknown')
                    points = props.get('overallPoints', 0)
                    team = props.get('teamName', 'Unknown')
                    print(f"  - {name}: {points} points (Team: {team})")
        except Exception as e:
            print(f"  - {manager.external_id}: (error reading properties: {e})")



Verifying data in CDF...
✓ Found 200 total nodes in fantasy_football space

Node breakdown:
  - Teams: 20
  - Gameweeks: 38
  - Managers: 27
  - Performance records: 115

✓ Manager details:


In [117]:
# Display performance records
if performance_nodes and manager_nodes:
    try:
        first_manager = manager_nodes[0]
        first_manager_id = first_manager.external_id
        
        # Get manager name
        manager_name = "Unknown"
        if hasattr(first_manager, 'properties') and first_manager.properties:
            props = first_manager.properties.get(SPACE, {}).get(MANAGER_VIEW, {}).get(VERSION, {})
            manager_name = props.get('managerName', 'Unknown')
        
        # Filter performance nodes for this manager
        manager_perf = [
            node for node in performance_nodes
            if first_manager_id in node.external_id
        ]
        
        if manager_perf:
            print(f"\n✓ Performance history for {manager_name} (showing first 5 gameweeks):")
            for perf in manager_perf[:5]:
                try:
                    if hasattr(perf, 'properties') and perf.properties:
                        props = perf.properties.get(SPACE, {}).get(PERFORMANCE_VIEW, {}).get(VERSION, {})
                        points = props.get('points', 0)
                        total = props.get('totalPoints', 0)
                        gw_num = perf.external_id.split('_')[-1].replace('gw', '')
                        print(f"  - Gameweek {gw_num}: {points} points (Total: {total})")
                except Exception as e:
                    print(f"  - {perf.external_id}: (error: {e})")
        else:
            print(f"\n✓ No performance records found for {manager_name}")
    except Exception as e:
        print(f"\nError displaying performance data: {e}")
else:
    print("\n(Skipping performance display - no data found)")



✓ No performance records found for Unknown


In [118]:
print("\n" + "="*80)
print("✅ SUCCESS! YOUR FANTASY FOOTBALL DATA IS NOW IN CDF!")
print("="*80)

print("\n📊 Data Summary:")
print(f"  ✓ Space: {SPACE}")
print(f"  ✓ Project: {client.config.project}")
print(f"  ✓ Total nodes loaded: {len(all_instances)}")
print(f"")
print(f"  Breakdown:")
print(f"    → {len(team_nodes)} Premier League Teams")
print(f"    → {len(gameweek_nodes)} Gameweeks")
print(f"    → {len(manager_nodes)} Managers from 'Hangover Hamre'")
print(f"    → {len(performance_nodes)} Performance records")

print("\n🎯 Your Managers:")
for i, node in enumerate(manager_nodes[:10], 1):
    print(f"  {i}. {node.external_id}")

print("\n" + "="*80)
print("🌐 VIEW YOUR DATA IN CDF FUSION")
print("="*80)
print(f"1. Go to: https://bluefield.cognitedata.com")
print(f"2. Navigate to: Data Modeling → Spaces")
print(f"3. Select space: {SPACE}")
print(f"4. Explore your views:")
print(f"   • Manager - See all your league members")
print(f"   • ManagerGameweekPerformance - Week-by-week stats")
print(f"   • Gameweek - Season schedule")
print(f"   • Team - Premier League teams")

print("\n🔄 KEEP YOUR DATA UPDATED")
print("="*80)
print("Run this notebook weekly to refresh with latest FPL data!")
print("Your data will be updated/upserted automatically.")

print("\n📈 NEXT STEPS")
print("="*80)
print("1. Explore data in CDF Fusion UI")
print("2. Use the 'explore_fpl_data.ipynb' notebook for analysis")
print("3. Create custom queries and visualizations")
print("4. Build dashboards in CDF")

print("\n" + "="*80)
print("🎉 Your Fantasy Football analytics platform is ready!")
print("="*80)



✅ SUCCESS! YOUR FANTASY FOOTBALL DATA IS NOW IN CDF!

📊 Data Summary:
  ✓ Space: fantasy_football
  ✓ Project: sofie-prod
  ✓ Total nodes loaded: 200

  Breakdown:
    → 20 Premier League Teams
    → 38 Gameweeks
    → 27 Managers from 'Hangover Hamre'
    → 115 Performance records

🎯 Your Managers:
  1. manager_653439
  2. manager_2589656
  3. manager_2078000
  4. manager_7470993
  5. manager_290121
  6. manager_7669946
  7. manager_4203952
  8. manager_3550350
  9. manager_4930626
  10. manager_631108

🌐 VIEW YOUR DATA IN CDF FUSION
1. Go to: https://bluefield.cognitedata.com
2. Navigate to: Data Modeling → Spaces
3. Select space: fantasy_football
4. Explore your views:
   • Manager - See all your league members
   • ManagerGameweekPerformance - Week-by-week stats
   • Gameweek - Season schedule
   • Team - Premier League teams

🔄 KEEP YOUR DATA UPDATED
Run this notebook weekly to refresh with latest FPL data!
Your data will be updated/upserted automatically.

📈 NEXT STEPS
1. Explor

In [109]:
# Check what views and data models exist in CDF
print("\n" + "="*80)
print("CHECKING CDF DATA MODELS & VIEWS")
print("="*80)

# Check spaces
print("\n1. Spaces in CDF:")
spaces = client.data_modeling.spaces.list(limit=50)
for space in spaces:
    if SPACE in space.space:
        print(f"   ✓ {space.space} - {space.name}")

# Check views
print("\n2. Views in fantasy_football space:")
try:
    views = client.data_modeling.views.list(space=SPACE, limit=50)
    if views:
        for view in views:
            print(f"   ✓ {view.external_id} (version {view.version})")
    else:
        print("   ⚠️  No views found - views may not have been deployed")
except Exception as e:
    print(f"   ⚠️  Error listing views: {e}")

# Check containers
print("\n3. Containers in fantasy_football space:")
try:
    containers = client.data_modeling.containers.list(space=SPACE, limit=50)
    if containers:
        for container in containers:
            print(f"   ✓ {container.external_id}")
    else:
        print("   ⚠️  No containers found")
except Exception as e:
    print(f"   ⚠️  Error listing containers: {e}")

# Check data models
print("\n4. Data Models:")
try:
    data_models = client.data_modeling.data_models.list(space=SPACE, limit=50)
    if data_models:
        for dm in data_models:
            print(f"   ✓ {dm.external_id} (version {dm.version})")
    else:
        print("   ⚠️  No data models found")
except Exception as e:
    print(f"   ⚠️  Error listing data models: {e}")

print("\n" + "="*80)



CHECKING CDF DATA MODELS & VIEWS

1. Spaces in CDF:
   ✓ fantasy_football - Fantasy Football

2. Views in fantasy_football space:
   ✓ Gameweek (version 1)
   ✓ Manager (version 1)
   ✓ ManagerGameweekPerformance (version 1)
   ✓ ManagerTeamBetting (version 1)
   ✓ Player (version 1)
   ✓ Team (version 1)
   ✓ Transfer (version 1)

3. Containers in fantasy_football space:
   ✓ Gameweek
   ✓ Manager
   ✓ ManagerGameweekPerformance
   ✓ ManagerTeamBetting
   ✓ Player
   ✓ Team
   ✓ Transfer

4. Data Models:
   ✓ FantasyFootball (version 2)



In [110]:
# Create the Data Model to group views together
# Note: This is idempotent - safe to run multiple times. It will create or update the data model.
# You only need to run this once, but it won't hurt to run it again.
from cognite.client.data_classes.data_modeling import DataModelApply, ViewId

print("\n" + "="*80)
print("ENSURING DATA MODEL EXISTS (with new views)")
print("="*80)

data_model = DataModelApply(
    space=SPACE,
    external_id="FantasyFootball",
    version="2",  # Increment version for new views
    name="Fantasy Football",
    description="Fantasy Premier League data model with enhanced analytics: managers, teams, players, transfers, and performance tracking",
    views=[
        ViewId(space=SPACE, external_id="Team", version="1"),
        ViewId(space=SPACE, external_id="Gameweek", version="1"),
        ViewId(space=SPACE, external_id="Manager", version="1"),
        ViewId(space=SPACE, external_id="ManagerGameweekPerformance", version="1"),
        ViewId(space=SPACE, external_id="Player", version="1"),
        ViewId(space=SPACE, external_id="Transfer", version="1"),
        ViewId(space=SPACE, external_id="ManagerTeamBetting", version="1"),
    ]
)

try:
    result = client.data_modeling.data_models.apply(data_model)
    print(f"✓ Data model ready: {result.external_id} (version {result.version})")
    print(f"  Contains {len(result.views)} views:")
    for view in result.views:
        print(f"    - {view.external_id}")
except Exception as e:
    print(f"⚠️  Error with data model: {e}")
    
print("="*80)



ENSURING DATA MODEL EXISTS (with new views)
✓ Data model ready: FantasyFootball (version 2)
  Contains 7 views:
    - Team
    - Gameweek
    - Manager
    - ManagerGameweekPerformance
    - Player
    - Transfer
    - ManagerTeamBetting


## Next Steps

Your Fantasy Football data is now in CDF! 🎉

### View in CDF Fusion:
1. Go to https://bluefield.cognitedata.com
2. Navigate to **Data Models** → `fantasy_football` space
3. Explore your data using the data model browser

### Query Examples:

```python
from cognite.client.data_classes.data_modeling.ids import ViewId

# Get all managers sorted by points
manager_view = ViewId(space="fantasy_football", external_id="Manager", version="1")
managers = client.data_modeling.instances.list(
    instance_type="node",
    sources=[manager_view],
    sort=[{"property": ["fantasy_football", "Manager/1", "overallPoints"], "direction": "descending"}]
)

# Get performance for specific gameweek
perf_view = ViewId(space="fantasy_football", external_id="ManagerGameweekPerformance", version="1")
gw12_performances = client.data_modeling.instances.list(
    instance_type="node",
    sources=[perf_view],
    filter={"equals": {"property": ["fantasy_football", "ManagerGameweekPerformance/1", "gameweek"], 
                      "value": {"space": "fantasy_football", "externalId": "gameweek_12"}}}
)
```

### Update Data:
Re-run this notebook anytime to refresh your data with the latest from FPL!
